In [36]:
import cv2
import mediapipe as mp
import json

In [37]:
labels_dataset = "./data/labels/1 label dataset.json"
values_dataset = "./data/values/1 values dataset.json"

In [38]:
landmark_list = json.load(open(values_dataset))
timestamps = [float(x) for x in list(landmark_list.keys())]

#loading labels from the key_log.txt file with format: timestamp,key
labels = json.load(open(labels_dataset))

In [39]:
len(labels), len(timestamps), len(landmark_list)

(229, 4796, 4796)

In [40]:
import numpy as np
values = []
for val in landmark_list.values():
    values.append(list(val.values()))

values = np.array(values)
values.shape

(4796, 21, 3)

In [58]:
data = []
window_size = 14
overlap = 3
for i in range(0, len(values) - window_size, overlap):
    data.append(values[i:i+window_size])

data = np.array(data)
data.shape

(1594, 14, 21, 3)

# Training autoencoder for self-supervised learning (easy way)
### This helps learning representation of dynamic gestures

In [59]:
import tensorflow as tf
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_shape = Input(shape=(window_size, 21, 3))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_shape)
x = MaxPooling2D((2, 3), padding='same')(x)
encoded = Conv2D(32, (3, 3), activation='relu', padding='same', name = "encoder")(x)
encoder_model = Model(input_shape, encoded)

decoder_input = Input(shape = (encoder_model.output_shape[1:]))
#x = Conv2D(32, (3, 3), activation='relu', padding='same')(decoder_input)
x = UpSampling2D((2, 3))(decoder_input)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((1, 1))(x)
decoded = Conv2D(3, (3, 3), activation='relu', padding='same')(x)
decoder_model = Model(decoder_input, decoded)

encoded_representation = encoder_model(input_shape)
decoded_output = decoder_model(encoded_representation)
autoencoder = Model(inputs=input_shape, outputs=decoded_output)

autoencoder.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 14, 21, 3)]       0         
                                                                 
 model_9 (Functional)        (None, 7, 7, 32)          5088      
                                                                 
 model_10 (Functional)       (None, 14, 21, 3)         5059      
                                                                 
Total params: 10147 (39.64 KB)
Trainable params: 10147 (39.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(data, data, epochs=30, batch_size=32, shuffle=True, validation_split=0.1)

Epoch 1/30


2023-06-20 17:27:39.948519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


45/45 [==============================] - 2s 25ms/step - loss: 0.3278 - val_loss: 0.1293
Epoch 2/30
 1/45 [..............................] - ETA: 0s - loss: 0.1891

2023-06-20 17:27:41.205154: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


45/45 [==============================] - 1s 16ms/step - loss: 0.1650 - val_loss: 0.0956
Epoch 3/30
45/45 [==============================] - 1s 14ms/step - loss: 0.1417 - val_loss: 0.0838
Epoch 4/30
45/45 [==============================] - 1s 14ms/step - loss: 0.1355 - val_loss: 0.0806
Epoch 5/30
45/45 [==============================] - 1s 14ms/step - loss: 0.1331 - val_loss: 0.0787
Epoch 6/30
45/45 [==============================] - 1s 15ms/step - loss: 0.1319 - val_loss: 0.0767
Epoch 7/30
45/45 [==============================] - 1s 14ms/step - loss: 0.1309 - val_loss: 0.0761
Epoch 8/30
45/45 [==============================] - 1s 15ms/step - loss: 0.1301 - val_loss: 0.0757
Epoch 9/30
45/45 [==============================] - 1s 14ms/step - loss: 0.1341 - val_loss: 0.0785
Epoch 10/30
45/45 [==============================] - 1s 13ms/step - loss: 0.1318 - val_loss: 0.0758
Epoch 11/30
45/45 [==============================] - 1s 14ms/step - loss: 0.1296 - val_loss: 0.0756
Epoch 12/30
45/45 [

# Finetuning the encoder part

In [61]:
#fine tuning the encoder model
from keras.models import Sequential
from keras.layers import Flatten
for layer in encoder_model.layers:
    layer.trainable = False

additional_layers = Sequential([
    Input(shape = (encoder_model.output_shape[1:])),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(4, activation='sigmoid')
]
)

In [62]:
model = Sequential([encoder_model, additional_layers])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_9 (Functional)        (None, 7, 7, 32)          5088      
                                                                 
 sequential_4 (Sequential)   (None, 4)                 873252    
                                                                 
Total params: 878340 (3.35 MB)
Trainable params: 873252 (3.33 MB)
Non-trainable params: 5088 (19.88 KB)
_________________________________________________________________


In [63]:
labels = json.load(open(labels_dataset, "r"))

#Now, from landmark_list, we want to extract a window of 0.5 seconds before and after each key press
#We will use the labels dictionary to label each window
#We will use the timestamps list to find the closest timestamp to the key press timestamp
#We will use the freq variable to calculate the number of frames we need to extract for each window
#We will use the loaded_landmark_list to extract the frames
window_size = 0.5  #0.5 seconds before and after the key press
freq = 14 #14 frames per second
window_frames = int(freq * window_size)
X, Y = [], []
for key in labels.keys():
    key = float(key)
    closest_timestamp = min(timestamps, key=lambda x:abs(x-key))
    index = timestamps.index(closest_timestamp)
    if index - window_frames < 0:
        continue
    if index + window_frames >= len(timestamps):
        continue
    
    window = []
    for i in range(index-window_frames, index+window_frames):
        k = str(timestamps[i])
        window.append(landmark_list[k])

    X.append(window)
    Y.append(labels[str(key)])

#retrive a structure with shape (n_windows, window_size, 21, 3)
data2 = []
for window in X:
    temp = []
    for sample in window:
        temp.append(np.array(list(sample.values())))
    
    data2.append(np.array(temp))

data = np.array(data2)
del data2


In [64]:
from sklearn.preprocessing import OneHotEncoder
#Y = [1 if y == 'Key.left' else 0 for y in Y]
Y = np.array(Y)
encoder = OneHotEncoder()
Y_encoded = encoder.fit_transform(Y.reshape(-1, 1)).toarray()

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, Y_encoded, test_size=0.2, random_state=42)

In [66]:
model.fit(X_train, y_train, epochs=30, batch_size=32, shuffle=True, validation_split=0.1)

Epoch 1/30
1/6 [====>.........................] - ETA: 2s - loss: 1.3746 - accuracy: 0.3125

2023-06-20 17:28:21.342919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 1s 63ms/step - loss: 0.8847 - accuracy: 0.7622 - val_loss: 0.6677 - val_accuracy: 0.8421
Epoch 2/30
4/6 [===================>..........] - ETA: 0s - loss: 0.4395 - accuracy: 0.9062

2023-06-20 17:28:21.740348: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 0s 22ms/step - loss: 0.4308 - accuracy: 0.8963 - val_loss: 0.3293 - val_accuracy: 0.8421
Epoch 3/30
6/6 [==============================] - 0s 23ms/step - loss: 0.3095 - accuracy: 0.9024 - val_loss: 0.2849 - val_accuracy: 0.8421
Epoch 4/30
6/6 [==============================] - 0s 23ms/step - loss: 0.3101 - accuracy: 0.9268 - val_loss: 0.1785 - val_accuracy: 0.8947
Epoch 5/30
6/6 [==============================] - 0s 20ms/step - loss: 0.2394 - accuracy: 0.9329 - val_loss: 0.1103 - val_accuracy: 0.9474
Epoch 6/30
6/6 [==============================] - 0s 20ms/step - loss: 0.2133 - accuracy: 0.9512 - val_loss: 0.0997 - val_accuracy: 1.0000
Epoch 7/30
6/6 [==============================] - 0s 19ms/step - loss: 0.1938 - accuracy: 0.9695 - val_loss: 0.0949 - val_accuracy: 1.0000
Epoch 8/30
6/6 [==============================] - 0s 20ms/step - loss: 0.1788 - accuracy: 0.9573 - val_loss: 0.0570 - val_accuracy: 1.0000
Epoch 9/30
6/6 [======================

In [67]:
y_pred = model.predict(X_test)

2/2 [==============================] - 0s 25ms/step


2023-06-20 17:28:27.559341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [68]:
y_test.shape, y_pred.shape

((46, 4), (46, 4))

In [70]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

array([[ 1,  0,  0,  0],
       [ 0, 14,  0,  0],
       [ 0,  0, 27,  0],
       [ 0,  0,  0,  4]])

In [69]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_test.argmax(axis = 1), y_pred.argmax(axis=1), average='weighted')
recall = recall_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
f1 = f1_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)

Precision:  1.0
Recall:  1.0
F1:  1.0


# Trying with baseline model

In [53]:
import keras
from keras.layers import Conv3D, MaxPooling2D, Flatten, Reshape, LSTM, Dense, Conv2D, Concatenate
num_classes = set(Y).__len__()
inp_shape = data.shape[1:]

# Input shape: (n_samples, 14, 21, 3)
input = Input(shape=inp_shape)
x = Conv2D(8, (2, 2), activation='relu', padding='same')(input)
x = MaxPooling2D((2, 3), padding='same')(x)
x = Conv2D(16, (2, 2), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 3), padding='same')(x)

y = Reshape((inp_shape[0]*inp_shape[1], inp_shape[2]))(input)
y = LSTM(16, return_sequences=True)(y)
y = Flatten()(y)
y = Dense(192, activation='relu')(y)
y = Reshape((4, 3, 16))(y)
y = Concatenate()([x, y])
y = Flatten()(y)
y = Dense(64, activation='relu')(y)
y = Dense(16, activation='relu')(y)
y = Dense(4, activation='softmax')(y)
model = Model(input, y)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=16, validation_split=0.1)

Epoch 1/30


2023-06-20 17:25:08.211723: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-20 17:25:08.395031: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 2/11 [====>.........................] - ETA: 0s - loss: 1.6577 - accuracy: 0.3125 

2023-06-20 17:25:08.600205: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - ETA: 0s - loss: 1.7204 - accuracy: 0.4878

2023-06-20 17:25:09.602691: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-20 17:25:09.680134: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 3s 109ms/step - loss: 1.7204 - accuracy: 0.4878 - val_loss: 1.5526 - val_accuracy: 0.8421
Epoch 2/30
11/11 [==============================] - 1s 48ms/step - loss: 0.7733 - accuracy: 0.8598 - val_loss: 0.5312 - val_accuracy: 0.8947
Epoch 3/30
11/11 [==============================] - 1s 49ms/step - loss: 0.4703 - accuracy: 0.9207 - val_loss: 0.3304 - val_accuracy: 0.8421
Epoch 4/30
11/11 [==============================] - 1s 48ms/step - loss: 0.3235 - accuracy: 0.9024 - val_loss: 0.2759 - val_accuracy: 0.8421
Epoch 5/30
11/11 [==============================] - 1s 46ms/step - loss: 0.3069 - accuracy: 0.9329 - val_loss: 0.2135 - val_accuracy: 0.9474
Epoch 6/30
11/11 [==============================] - 1s 47ms/step - loss: 0.2903 - accuracy: 0.9024 - val_loss: 0.2487 - val_accuracy: 0.8421
Epoch 7/30
11/11 [==============================] - 1s 46ms/step - loss: 0.2647 - accuracy: 0.9207 - val_loss: 0.2002 - val_accuracy: 0.8947
Epoch 8/30
11/11 [=====

In [54]:
y_pred = model.predict(X_test)

2/2 [==============================] - 0s 39ms/step


2023-06-20 17:25:27.982972: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-20 17:25:28.043809: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [55]:
precision = precision_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
recall = recall_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
f1 = f1_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average='weighted')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)


Precision:  0.968167701863354
Recall:  0.9565217391304348
F1:  0.9530961791831356


In [25]:
6264
5823341

5823341